<a href="https://colab.research.google.com/github/erickkkt/AIServices/blob/master/bot_crawl_batdongsan_firebase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#firebase
pip install firebase_admin

SyntaxError: ignored

In [ ]:
pip install beautifulsoup4

In [ ]:
pip install lxml

In [ ]:
pip install html5lib

In [ ]:
pip install requests

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
#import
import uuid
import requests
import os
import re
import firebase_admin
from firebase_admin import db
from firebase_admin import credentials, firestore
from bs4 import BeautifulSoup

In [ ]:
#test drive
os.listdir('/content/drive/My Drive')

['How to get started with Drive.pdf',
 'Dataset',
 'AI_COLAB',
 'HDA',
 'Colab Notebooks',
 'firebase',
 'win10_emulator_snap.img.lz4']

In [ ]:
#init firebase 
cred = credentials.Certificate('/content/drive/My Drive/firebase/bdsvn-2e71f-firebase-adminsdk-8w4f2-c48c4c298c.json')
firebase_admin.initialize_app(cred)

In [ ]:
#init database node
db = firestore.client()

In [ ]:
 headers = {
    'Accept-Encoding': 'gzip, deflate, sdch',
    'Accept-Language': 'en-US,en;q=0.8',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Cache-Control': 'max-age=0',
    'Connection': 'keep-alive',
  }

In [ ]:
BASE_URL = 'https://batdongsan.com.vn'

In [ ]:
def crawl_page(url, presentImage=''):
  result = requests.get(url, headers=headers, allow_redirects=False)
  print(result.status_code)
  content = result.content
  soup = BeautifulSoup(content, 'html.parser')
  # print(soup)
  descriptions = soup.find_all('div', {"class":"container-default"})
  container = descriptions[0]
  # print(container)
  titles = container.find_all('h1')
  title = titles[0].text.replace('\r','').replace('\n','').strip()
  chitiet = container.find('div', {"class":"kqchitiet"})
  
  # showImage = container.find('div', {"class":"show-img"})
  # displayImage = showImage.find('img', {"id":"imgSlide1"})

  # medias
  medias = []
  slider = container.find('ul',{"id":"thumbs"})
  images = slider.find_all('img')
  for image in images:
    medias.append(image['src'].replace('200x200','745x510'))

  #extend
  extend = []
  table1 = container.find('div', {"class":"table-detail", "id":"product-other-detail"})

  rows1 = table1.find_all('div', {"class":"row"})
  for row in rows1:
    left = row.find('div', {"class":"left"})
    leftT = left.text.lower().replace('\r','').replace('\n','').strip()    
    right = row.find('div', {"class":"right"})
    rightT = right.text.lower().replace('\r','').replace('\n','').strip()    
    obj = {leftT:rightT}
    extend.append(obj)

  #contact
  brokerInfo = []
  table2 = container.find('div', {"class":"div-table-cell table2"})

  avartarImg = table2.find('img')
  if avartarImg is not None:
    brokerInfo.append({'avartar':avartarImg['src']})

  rows2 = table2.find_all('div', {"class":"right-content"})
  for row in rows2:
    left = row.find('div', {"class":"left"})
    leftT = left.text.lower().replace('\r','').replace('\n','').strip()    
    right = row.find('div', {"class":"right"})
    rightT = right.text.lower().replace('\r','').replace('\n','').strip()    
    obj = {leftT:rightT}
    brokerInfo.append(obj)

  #up dateS
  moreInfo = container.find('div', {"class":"prd-more-info"})
  upDateDiv = moreInfo.find('div',{'style':'width: 166px'})
  upDs = upDateDiv.text.split(':')
  upDate = upDs[1].lower().replace('\r','').replace('\n','').strip()

  #address, long, lat
  latInput = container.find('input', {"id":"hdLat"})
  lat = latInput['value']

  longInput = container.find('input', {"id":"hdLong"})
  long = longInput['value']

  location = {'long':long, 'lat':lat}
  addressInput = container.find('input', {"id":"hdAddress"})
  address=addressInput['value']


  #details
  spans = chitiet.find_all('span')
  kv = ''
  gia = ''
  dt = ''
  for s in spans:
    text = s.text.lower().replace('\r','').replace('\n','').strip()
    splited = text.split(':')
    if len(splited) == 2:
      if splited[0].strip() == 'khu vực':
        kv = splited[1].strip()
        continue
      if splited[0].strip() == 'giá':
        gia = splited[1].strip()
        continue
      if splited[0].strip() == 'diện tích':
        dt = splited[1].strip()
        continue
  mota = container.find('div', {"class":"pm-desc"})
  aaa = mota.text.strip().split('\n')
  mt = aaa[0]
  return {'title': title, 'presentImage': presentImage, 'postedDate':upDate, 'address': address, 'location':location, 'link':url, 'area':kv, 'price':gia, 'square':dt, 'description':str(mota), 'medias':medias, 'extend':extend, 'contact':brokerInfo}

In [ ]:
def save_to_fire_base(collection, data):
  db.collection(collection).document(str(uuid.uuid4())).set(data)

In [ ]:
def crawl_category_page(saleType, collection, page=''):
  url = BASE_URL +'/'+ saleType
  if page:
    url = url + '/' + page

  result = requests.get(url, headers=headers, allow_redirects=False)
  print(result.status_code)
  content = result.content
  soup = BeautifulSoup(content, 'html.parser')
  # print(soup)
  products = soup.find_all('div', {"class":"search-productItem"})

  for product in products:
    avatar = product.find('a',{"class":"product-avatar"})
    link = BASE_URL + avatar['href']
    print(link)
    image = product.find('img',{"class":"product-avatar-img"})
    presentImage = image['src']
    try:      
      data = crawl_page(link, presentImage)
      save_to_fire_base(collection, data)
    except:
      print("An exception occurred")
    
  print('done')

In [ ]:
def crawl_category(saleType):
  url = BASE_URL +'/'+ saleType
  print(url)
  result = requests.get(url, headers=headers, allow_redirects=False)
  print(result.status_code)
  content = result.content
  soup = BeautifulSoup(content, 'html.parser')
  footer = soup.find('div',{'class':'Footer'})
  txt = footer.text.lower().replace('\r','').replace('\n','').replace(',','').strip()
  numbers = re.findall(r"[-+]?\d*\.\d+|\d+", txt)
  pages = int(int(numbers[0])/20) + 1
  # crawl_category_page('ban-nha-rieng', 'ban-nha-rieng')
  for x in range(1, pages):
    if x == 1:
      crawl_category_page('ban-nha-rieng', 'ban-nha-rieng')
    else:
      p = 'p{}'.format(x)
      print(p)
      crawl_category_page('ban-nha-rieng', 'ban-nha-rieng', p)


In [ ]:
# url = 'https://batdongsan.com.vn/ban-nha-rieng-thu-duc/p2'
url = 'https://batdongsan.com.vn/ban-nha-rieng-duong-ben-me-coc-phuong-15-2/pho-quan-8-4-tam-cach-buu-dien-q-5-4km-gia-chi-1-75-ty-0902331665-pr24976152'
a = crawl_page(url, 'https://file4.batdongsan.com.vn/crop/350x232/2020/04/05/20200405133640-4bc1_wm.jpg')
print(a)
save_to_fire_base('ban-nha-rieng', a)

200
{'title': 'BÁN NHÀ PHỐ QUẬN 8, 4 TẤM, CÁCH BƯU ĐIỆN Q. 5 4KM, GIÁ CHỈ 1.7 TỶ, 0902331665', 'presentImage': 'https://file4.batdongsan.com.vn/crop/350x232/2020/04/05/20200405133640-4bc1_wm.jpg', 'postedDate': '29-05-2020', 'address': 'Đường Bến Mễ Cốc, Phường 15, Quận 8, Hồ Chí Minh', 'location': {'long': '106.6332948', 'lat': '10.7257399'}, 'link': 'https://batdongsan.com.vn/ban-nha-rieng-duong-ben-me-coc-phuong-15-2/pho-quan-8-4-tam-cach-buu-dien-q-5-4km-gia-chi-1-75-ty-0902331665-pr24976152', 'area': 'bán nhà riêng tại đường bến mễ cốc - quận 8 - hồ chí minh', 'price': '1.7 tỷ', 'square': '22.4m²', 'description': '<div class="pm-desc">\r\nBán nhà phố quận 8, 4 tấm, cách Bưu Điện Q. 5 4km, giá chỉ 1.7 tỷ, 0902331665.<br/>- DTXD: 3.2mx7m.<br/>- DTSD: 84m2.<br/>- 1PK + 2PN + 3WC - trệt 2 lầu 1 sân thượng - Kiến trúc hiện đại - nội thất cực kì cao cấp và hiện đại.<br/><br/>- Giấy tờ đầy đủ, đi công chứng trong ngày.<br/>- Vị trí rất đẹp, gần chợ, trường học, BV...<br/>- Tiện ở, cho th

In [ ]:
crawl_category('ban-nha-rieng')


https://batdongsan.com.vn/ban-nha-rieng
200
200
https://batdongsan.com.vn/ban-nha-rieng-binh-chanh/mo-biet-thu-khu-do-thi-an-phu-villa-re-nhat-khu-vuc-nam-ngay-mt-tran-van-giau-chanh-pr25686452
200
https://batdongsan.com.vn/ban-nha-rieng-phuong-phan-chu-trinh/chinh-can-xay-1-tang-1-gac-lung-trinh-pr25681108
200
https://batdongsan.com.vn/ban-nha-rieng-duong-nguyen-trai-phuong-khuong-trung/pho-mat-ngo-oto-mt-6m-kinh-doanh-dinh-so-vuong-3-mat-thoang-pr25679455
200
https://batdongsan.com.vn/ban-nha-rieng-xa-duc-lap-thuong/gap-lo-dat-612m2-dat-tho-cu-100-huyen-hoa-tinh-long-an-pr25672488
200
https://batdongsan.com.vn/ban-nha-rieng-duong-so-10-phuong-linh-chieu/gap-chinh-chu-trung-tam-quan-thu-duc-pr25664248
200
https://batdongsan.com.vn/ban-nha-rieng-phuong-trang-dai-prj-khu-dan-cu-phu-gia/so-tho-bien-hoa-dong-naiq-pr25290451
200
https://batdongsan.com.vn/ban-nha-rieng-duong-quoc-lo-50-xa-quy-duc/chinh-chu-can-nhieu-moi-xay-ql-50-giap-ranh-binh-chanh-tu-650tr-chinh-chu-da-ra-so-pr25233929
2